In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import os
import glob
import re
import seaborn as sns

import sys
sys.path.append("..")
from data_preparation.data_preparation_pos import ABSATokenizer, convert_examples_to_tf_dataset, read_conll
import utils.utils as utils
import utils.pos_utils as pos_utils
import utils.postprocessing_utils as post_utils

### PoS

In [19]:
pos = post_utils.retrieve_results("../results/results_pos_postprocessed.xlsx", 3)["Accuracy"]["langvlang"].iloc[:15, :-1]

pos = pos.replace("-", np.nan)

In [20]:
pos = pos.rename(columns={"Test\Train": "Train Language", "Group": "Train Group"})

In [21]:
pos = pd.melt(pos, id_vars=["Train Group", "Train Language"], value_vars=pos.columns[2:], var_name="Test Language", value_name="Score")

In [22]:
pos["Score"] = pos["Score"] * 100

In [23]:
lang_to_group = utils.make_lang_group_dict()

In [24]:
pos["Test Group"] = pos["Test Language"].apply(lambda x: lang_to_group[x])

In [25]:
within = pos[pos["Train Language"] == pos["Test Language"]][["Train Language", "Score"]]

In [26]:
pos = pos[pos["Train Language"] != pos["Test Language"]]

In [27]:
pos = pd.merge(pos, within, on="Train Language", how="left").rename(columns={"Score_x": "Cross", "Score_y": "Within"})

In [28]:
pos["Transfer"] = pos["Within"] - pos["Cross"]

In [29]:
pos["Group Difference"] = pos[["Train Group", "Test Group"]].apply(
    lambda x: "Same" if x.values[0] == x.values[1] else "Other", axis=1
)

In [31]:
#pos.to_excel("../results/melted_results_pos.xlsx", index=False)

### Sentiment

In [32]:
sentiment = post_utils.retrieve_results("../results/results_sentiment_postprocessed.xlsx", 3)["Accuracy"]["langvlang"].iloc[:12, :-1]

In [33]:
sentiment = sentiment.rename(columns={"Test\Train": "Train Language", "Group": "Train Group"})

In [34]:
sentiment = pd.melt(sentiment, id_vars=["Train Group", "Train Language"], value_vars=sentiment.columns[2:], var_name="Test Language", value_name="Score")

In [35]:
lang_to_group = utils.make_lang_group_dict()

In [36]:
sentiment["Test Group"] = sentiment["Test Language"].apply(lambda x: lang_to_group[x])

In [37]:
within = sentiment[sentiment["Train Language"] == sentiment["Test Language"]][["Train Language", "Score"]]

In [38]:
sentiment = sentiment[sentiment["Train Language"] != sentiment["Test Language"]]

In [39]:
sentiment = pd.merge(sentiment, within, on="Train Language", how="left").rename(columns={"Score_x": "Cross", "Score_y": "Within"})

In [40]:
sentiment["Transfer"] = sentiment["Within"] - sentiment["Cross"]

In [41]:
sentiment["Group Difference"] = sentiment[["Train Group", "Test Group"]].apply(
    lambda x: "Same" if x.values[0] == x.values[1] else "Other", axis=1
)

In [42]:
#sentiment.to_excel("../results/melted_results_sentiment.xlsx", index=False)